In [ ]:
!pip install requests beautifulsoup4 pandas tqdm


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
from tqdm import tqdm

BASE_URL = "https://books.toscrape.com/catalogue/"
START_PAGE = "page-1.html"

products = []
product_id = 1

current_page = START_PAGE

while current_page:
    url = BASE_URL + current_page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    items = soup.find_all("article", class_="product_pod")

    for item in items:
        product_name = item.h3.a["title"]
        price_text = item.find("p", class_="price_color").text

        price = re.sub(r'[^\d.]', '', price_text)
        availability = item.find("p", class_="instock availability").text.strip()
        rating = item.p["class"][1]
        product_url = BASE_URL + item.h3.a["href"]

        products.append({
            "product_id": product_id,
            "product_name": product_name,
            "category": "Books",
            "price": float(price),
            "rating": rating,
            "availability": availability,
            "number_of_reviews": None,
            "product_url": product_url,
            "scrape_date": datetime.today().date()
        })

        product_id += 1


    next_button = soup.find("li", class_="next")
    if next_button:
        current_page = next_button.a["href"]
    else:
        current_page = None

print(f"Total scraped products: {len(products)}")

df = pd.DataFrame(products)
df.to_csv("books_dataset.csv", index=False)

df.head()
